### Write json file for plip output

In [2]:
import pandas as pd
import json
import requests

from pyspark.sql.types import ArrayType, StringType, IntegerType, StructType, StructField
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

# establish spark connection
spark = (
    SparkSession.builder
    .master('local[*]')
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/09 11:05:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pd.set_option('display.max_rows', 100)

In [4]:
df = (
    spark.read.csv('output.csv', sep=',', header=True)
    # Somehow there are duplications:
    .distinct() 
    .groupBy(['pdb_structure_id', 'compound_id', 'prot_residue_number','prot_chain_id', 'prot_residue_type'])
    .agg(
        f.collect_set(f.col('interaction_type')).alias('interaction_types')
    )
)

In [5]:
# df.write.json('plip_output_aggregated.json')

In [6]:
# %%bash 

# cat /Users/dsuveges/project_data/marine/plip_output_aggregated.json/*json \
#     | gzip > /Users/dsuveges/project_data/marine/plip_output_aggregated.json.gz

In [48]:
import pandas as pd
from functools import reduce

# 319408 -> unique 265603
df = (
    pd.read_json('plip_output_aggregated.json.gz', orient='records', lines=True)
)

df.head()

,pdb_structure_id,compound_id,prot_residue_number,prot_chain_id,prot_residue_type,interaction_types
0,11gs,GSH,52,A,LEU,[hbond]
1,13gs,GSH,52,A,LEU,[hbond]
2,13gs,SAS,10,A,VAL,[hydroph_interaction]
3,13gs,SAS,13,A,ARG,[saltbridge]
4,13gs,SAS,35,B,VAL,[hbond]


In [107]:
df_sample = df.sample(frac=0.000015)

In [108]:
len(df_sample)

4

### Group by pdb structure and take only one group (one structure)

### The function to get uniprot residue position and Uniprot accession

In [109]:
def get_pdb_sifts_mapping(pdb_id: str) -> pd.DataFrame:
    URL = f'https://www.ebi.ac.uk/pdbe/graph-api/mappings/ensembl/{pdb_id}'
    data = requests.get(URL).json()

    return (
        pd.DataFrame(reduce(lambda x,y: x + y['mappings'], data[pdb_id]['Ensembl'].values(), []))
        .assign(
            author_start = lambda df: df.start.apply(lambda start: start['author_residue_number']),
            author_end = lambda df: df.end.apply(lambda end: end['author_residue_number']),
            uniprot_position = lambda df: df.apply(lambda row: list(range(row['unp_start'], row['unp_end']+1)), axis=1),
            diff = lambda df: df.apply(lambda row: row['author_start'] - row['unp_start'], axis=1)
        )
        .explode('uniprot_position')
        .assign(
            prot_residue_number = lambda df: df.apply(lambda row: row['uniprot_position'] + row['diff'], axis=1)
        )
        [['accession', 'chain_id', 'uniprot_position', 'prot_residue_number']]
        .rename(columns={'chain_id': 'prot_chain_id'})
        .drop_duplicates()
    )


def map2uniprot(plip_df: pd.DataFrame) -> pd.DataFrame:
    # Extracting pdb identifier:
    pdb_id = plip_df.pdb_structure_id.iloc[0]
    
    # Fetch mappings from pdb api:
    sifts_df = get_pdb_sifts_mapping(pdb_id)
    
    # Join with mapping:
    return (
        plip_df
        .merge(sifts_df, on=['prot_chain_id', 'prot_residue_number'], how='left')
    )


# test_df = grouped.get_group('3e7g')
# map2uniprot(test_df)
# get_pdb_sifts_mapping('3e7g')

grouped = (
    df_sample
    .groupby('pdb_structure_id')
)


grouped.apply(map2uniprot).reset_index(drop=True)

,pdb_structure_id,compound_id,prot_residue_number,prot_chain_id,prot_residue_type,interaction_types,accession,uniprot_position
0,3q06,ZN,238,A,CYS,[metal_complex],P04637-9,106
1,3q06,ZN,238,A,CYS,[metal_complex],P04637-8,106
2,3q06,ZN,238,A,CYS,[metal_complex],P04637-5,199
3,3q06,ZN,238,A,CYS,[metal_complex],P04637-4,199
4,3q06,ZN,238,A,CYS,[metal_complex],P04637-3,238
5,3q06,ZN,238,A,CYS,[metal_complex],P04637-2,238
6,3q06,ZN,238,A,CYS,[metal_complex],P04637-7,106
7,3q06,ZN,238,A,CYS,[metal_complex],P04637-6,199
8,3q06,ZN,238,A,CYS,[metal_complex],PRO_0000185703,238
9,3q06,ZN,238,A,CYS,[metal_complex],P04637,238


In [ ]:
# def map2uniprot(plip_df: pd.DataFrame) -> pd.DataFrame:
#     pdb_id = plip_df.pdb_id.iloc

# pdb_id = test_df.pdb_structure_id.iloc[0]

# sifts_df.append(sifts_df, ignore_index=True)
# sifts_df = get_pdb_sifts_mapping(pdb_id)
# sifts_df.head()

### Merge the output of the function "sifts_df" with the input of the fuction (one group) to obtain the other infos on the residue

In [20]:
le_gros_df_sp.count()

19

In [46]:
residue_pos_df = (
    test_df
    .merge(sifts_df, on=['prot_chain_id', 'prot_residue_number'], how='left')
)
residue_pos_df

,pdb_structure_id,compound_id,prot_residue_number,prot_chain_id,prot_residue_type,interaction_types,accession,uniprot_position,author_start,author_end,unp_start,unp_end
0,3tdk,NAD,37,B,VAL,[hbond],O60701-2,37,1.0,54.0,1.0,54.0
1,3tdk,NAD,37,B,VAL,[hbond],PRO_0000074060,37,1.0,54.0,1.0,54.0
2,3tdk,NAD,37,B,VAL,[hbond],O60701,37,1.0,54.0,1.0,54.0
3,4d0m,GSP,156,B,LEU,[hbond],O60701-3,59,156.0,221.0,59.0,124.0
4,4d0m,GSP,156,B,LEU,[hbond],PRO_0000074060,156,156.0,221.0,156.0,221.0
5,4d0m,GSP,156,B,LEU,[hbond],O60701,156,156.0,221.0,156.0,221.0
6,6c4j,NAD,161,L,GLU,[hydroph_interaction],O60701-3,64,156.0,221.0,59.0,124.0
7,6c4j,NAD,161,L,GLU,[hydroph_interaction],PRO_0000074060,161,156.0,221.0,156.0,221.0
8,6c4j,NAD,161,L,GLU,[hydroph_interaction],O60701,161,156.0,221.0,156.0,221.0
9,6c4j,NAD,161,L,GLU,[hydroph_interaction],O60701-2,94,157.0,221.0,90.0,154.0


In [22]:
len(residue_pos_df)

38

### Open the crossref Uniprot Ensembl to add the Ensembl id

In [23]:
schema = StructType([
    StructField("UniProtKB-AC", StringType(), True),
    StructField("ID_type", StringType(), True),
    StructField("ID", StringType(), True)
    ])

cross_ref_uniprot = (
                    spark
                        .read.csv("../cross_ref_uniprot_ensembl_prot/HUMAN_9606_idmapping.tsv", sep="\t", schema=schema)
                        .filter(f.col('ID_type').rlike('Ensembl_PRO'))
                        .select(f.col('UniProtKB-AC'), f.col('ID'))
                        .withColumnRenamed("UniProtKB-AC", "accession")
                        .withColumnRenamed("ID", "ensemblProtId")
                        .toPandas()
    )
# 6289868 rows full dataset
# 115247 rows filtered dataset
cross_ref_uniprot
# Splice variants (different UTR regions but same sequence so, should be the same genomic location)

,accession,ensemblProtId
0,P31946,ENSP00000300161
1,P31946,ENSP00000361930
2,P62258,ENSP00000264335
3,P62258-2,ENSP00000461762
4,P62258-2,ENSP00000481059
...,...,...
115242,E9PPE7,ENSP00000432733
115243,E9PRZ4,ENSP00000432213
115244,A0A5F9ZHV4,ENSP00000500622
115245,A0A7P0T8F6,ENSP00000505116


### Merge the crossref df with the residue position df on Uniprot id

In [24]:
residue_pos_df = (
    cross_ref_uniprot
    .merge(residue_pos_df, on='accession', how='inner')
    # .drop(["accession"])
)
residue_pos_df.head()

,accession,ensemblProtId,pdb_structure_id,compound_id,prot_residue_number,prot_chain_id,prot_residue_type,interaction_types,uniprot_position,author_start,author_end,unp_start,unp_end
0,P09211,ENSP00000381607,11gs,GSH,52,A,LEU,[hbond],53,48,77,49,78
1,P09211,ENSP00000381607,11gs,GSH,13,B,ARG,"[hydroph_interaction, saltbridge]",14,12,47,13,48
2,P09211,ENSP00000381607,11gs,GSH,7,B,TYR,[halogenbond],8,0,12,1,13
3,P09211,ENSP00000381607,11gs,GSH,44,A,LYS,[saltbridge],45,12,47,13,48
4,P09211,ENSP00000381607,11gs,GSH,51,A,GLN,[hbond],52,48,77,49,78


### Open the mapping residue index to genomic position file

In [25]:
generated_mapping = (
    pd.read_csv("residue_gen_pos_output/generated_mappings_2.tsv", sep="\t")
    .rename(columns={"protein_id": "ensemblProtId", "gene_id": "geneId", "amino_acid_position": "prot_residue_number"})
)
generated_mapping

/var/folders/sq/c1y59gj1251c4t6tm79lcrlc0000gn/T/ipykernel_75874/2376170891.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("residue_gen_pos_output/generated_mappings_2.tsv", sep="\t")


,pos1,pos2,pos3,ensemblProtId,geneId,chr,strand,prot_residue_number
0,127588499,127588500,127588501,ENSP00000000233,ENSG00000004059,7,+,1
1,127588502,127588503,127588504,ENSP00000000233,ENSG00000004059,7,+,2
2,127588505,127588506,127588507,ENSP00000000233,ENSG00000004059,7,+,3
3,127588508,127588509,127588510,ENSP00000000233,ENSG00000004059,7,+,4
4,127588511,127588512,127588513,ENSP00000000233,ENSG00000004059,7,+,5
...,...,...,...,...,...,...,...,...
42439476,22731559,22731560,22731561,ENSP00000512964,ENSG00000136244,7,+,263
42439477,22731562,22731563,22731564,ENSP00000512964,ENSG00000136244,7,+,264
42439478,22731565,22731566,22731567,ENSP00000512964,ENSG00000136244,7,+,265
42439479,22731568,22731569,22731570,ENSP00000512964,ENSG00000136244,7,+,266


### Merge the mapping residue to genomic position DF WITH the residue position with the good ensembl protein id DF

In [26]:
residue_gen_pos_df = (
    residue_pos_df
    .merge(generated_mapping, on=['ensemblProtId', 'prot_residue_number'], how='inner')
    # .drop(["accession"])
)
residue_gen_pos_df

,accession,ensemblProtId,pdb_structure_id,compound_id,prot_residue_number,prot_chain_id,prot_residue_type,interaction_types,uniprot_position,author_start,author_end,unp_start,unp_end,pos1,pos2,pos3,geneId,chr,strand
0,P09211,ENSP00000381607,11gs,GSH,52,A,LEU,[hbond],53,48,77,49,78,67584694,67584695,67584696,ENSG00000084207,11,+
1,P09211,ENSP00000381607,11gs,GSH,52,B,LEU,[hbond],53,48,77,49,78,67584694,67584695,67584696,ENSG00000084207,11,+
2,P09211,ENSP00000381607,11gs,GSH,13,B,ARG,"[hydroph_interaction, saltbridge]",14,12,47,13,48,67584169,67584464,67584465,ENSG00000084207,11,+
3,P09211,ENSP00000381607,11gs,GSH,13,A,ARG,"[hydroph_interaction, saltbridge]",14,12,47,13,48,67584169,67584464,67584465,ENSG00000084207,11,+
4,P09211,ENSP00000381607,11gs,GSH,7,B,TYR,[halogenbond],8,0,12,1,13,67584151,67584152,67584153,ENSG00000084207,11,+
5,P09211,ENSP00000381607,11gs,GSH,7,A,TYR,[halogenbond],8,0,12,1,13,67584151,67584152,67584153,ENSG00000084207,11,+
6,P09211,ENSP00000381607,11gs,GSH,44,A,LYS,[saltbridge],45,12,47,13,48,67584556,67584557,67584558,ENSG00000084207,11,+
7,P09211,ENSP00000381607,11gs,GSH,44,B,LYS,[saltbridge],45,12,47,13,48,67584556,67584557,67584558,ENSG00000084207,11,+
8,P09211,ENSP00000381607,11gs,GSH,51,A,GLN,[hbond],52,48,77,49,78,67584691,67584692,67584693,ENSG00000084207,11,+
9,P09211,ENSP00000381607,11gs,GSH,51,B,GLN,"[hydroph_interaction, hbond]",52,48,77,49,78,67584691,67584692,67584693,ENSG00000084207,11,+


In [30]:
def fetch_residue(row: pd.Series) -> str:
    """Fetch amnino acid from Ensembl based on the translation id and uniprot_position"""
    
    pos = row['uniprot_position']
    tid = row['ensemblProtId']

    URL = f'https://rest.ensembl.org/sequence/id/{tid}?content-type=text/plain&start={pos}&end={pos}'
    try:
        return requests.get(URL).text
    except:
        return None

In [36]:
residue_gen_pos_df['aa_check'] = residue_gen_pos_df.apply(fetch_residue, axis=1)
(
    residue_gen_pos_df
    .drop(["accession", "interaction_types", "author_start", "author_end", "unp_start", "unp_end", "pos1", "pos2", "pos3"], axis=1)
 )

,ensemblProtId,pdb_structure_id,compound_id,prot_residue_number,prot_chain_id,prot_residue_type,uniprot_position,geneId,chr,strand,aa_check
0,ENSP00000381607,11gs,GSH,52,A,LEU,53,ENSG00000084207,11,+,L
1,ENSP00000381607,11gs,GSH,52,B,LEU,53,ENSG00000084207,11,+,L
2,ENSP00000381607,11gs,GSH,13,B,ARG,14,ENSG00000084207,11,+,R
3,ENSP00000381607,11gs,GSH,13,A,ARG,14,ENSG00000084207,11,+,R
4,ENSP00000381607,11gs,GSH,7,B,TYR,8,ENSG00000084207,11,+,Y
5,ENSP00000381607,11gs,GSH,7,A,TYR,8,ENSG00000084207,11,+,Y
6,ENSP00000381607,11gs,GSH,44,A,LYS,45,ENSG00000084207,11,+,K
7,ENSP00000381607,11gs,GSH,44,B,LYS,45,ENSG00000084207,11,+,K
8,ENSP00000381607,11gs,GSH,51,A,GLN,52,ENSG00000084207,11,+,Q
9,ENSP00000381607,11gs,GSH,51,B,GLN,52,ENSG00000084207,11,+,Q


22/05/06 00:15:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 838993 ms exceeds timeout 120000 ms
22/05/06 00:15:10 WARN SparkContext: Killing executors is not supported by current scheduler.
